# 네이버 뉴스 크롤 만들기
date: 2024-01-17
written by: [Jehwan Kim](github.com/kreimben)

In [11]:
# !pip install -r requirements.txt --upgrade

In [12]:
import traceback
from datetime import datetime, timedelta
from time import sleep

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [13]:
end_date = datetime.now()
start_date = end_date - timedelta(days=2)  # 최근 3일간의 데이터를 위한 설정

end_date = end_date.strftime("%Y%m%d")
start_date = start_date.strftime("%Y%m%d")

query = '기사'
url = f"https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={start_date}&de={end_date}&docid=&nso=so:r,p:from{start_date}to{end_date},a:all&mynews=0&refresh_start=0&related=0"
max_page = 100  # 크롤링을 원하는 최대 페이지 수 지정

start_date, end_date

('20240115', '20240117')

In [14]:
# 각 기사들의 데이터를 종류별로 나눠담을 리스트를 생성합니다. (추후 DataFrame으로 모을 예정)
titles = []
dates = []
articles = []
article_urls = []
press_companies = []
categories = []
category_kind = {
    '정치': 100, '경제': 101, '사회': 102, '생활/문화': 103, "세계": 104, "IT/과학": 105, '연예': 106, '스포츠': 107
}

# 지정한 기간 내 원하는 페이지 수만큼의 기사를 크롤링합니다.
current_call = 1
last_call = (max_page - 1) * 10 + 1  # max_page이 5일 경우 41에 해당 (1페이지는 url에 포함되어 있으므로 1을 빼줌)

# For error rate calulation
errors = []

# For exclude duplicated articles
visit = []

In [15]:
while current_call <= last_call:

    print('\n{}번째 기사글부터 크롤링을 시작합니다.'.format(current_call))

    url = "https://search.naver.com/search.naver?where=news&query=" + query \
          + "&nso=so%3Ar%2Cp%3Afrom" + start_date \
          + "to" + end_date \
          + "%2Ca%3A&start=" + str(current_call)

    web = requests.get(url).content
    source = BeautifulSoup(web, 'html.parser')

    urls_list = []
    for urls in source.find_all('a', {'class': "info"}):
        if urls["href"].startswith("https://n.news.naver.com"):
            urls_list.append(urls["href"])

    for url in urls_list:
        # 중복 기사 제거
        if url in visit:
            continue
        else:
            visit.append(url)

        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
            web_news = requests.get(url, headers=headers).content
            source_news = BeautifulSoup(web_news, 'html.parser')

            if title := source_news.find('h2', {'class': 'media_end_head_headline'}):
                title = title.get_text()
            else:
                title = source_news.find('h2', {'class': 'end_tit'}).get_text()
            print('Processing article : {}'.format(title))

            date = source_news.find('span', {'class': 'media_end_head_info_datestamp_time'}).get_text()

            article = source_news.find('article', {'id': 'dic_area'}).get_text()
            article = article.replace("\n", "")
            article = article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
            article = article.replace("동영상 뉴스       ", "")
            article = article.replace("동영상 뉴스", "")
            article = article.strip()

            press_company = source_news.find('em', {'class': 'media_end_linked_more_point'}).get_text()

            titles.append(title)
            dates.append(date)
            articles.append(article)
            press_companies.append(press_company)
            article_urls.append(url)

            for k, v in category_kind.items():
                find = f'sid={v}'
                target = url
                if find in target:
                    categories.append(k)
                    break
            else:
                categories.append('N/A')
        except Exception as e:
            print(f'*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : {url}')
            print(f'에러 내용 : {e}')
            print(traceback.format_exc())
            errors.append(url)

    # 대량의 데이터를 대상으로 크롤링을 할 때에는 요청 사이에 쉬어주는 타이밍을 넣는 것이 좋습니다.
    sleep(1)
    current_call += 10


1번째 기사글부터 크롤링을 시작합니다.
Processing article : 전청조 ‘공범’ 주장에…남현희 “사기꾼 말 기사화 그만, 억울하다”
Processing article : 온 힘 다해 큰불 막은 택배기사, 소방차 오자 재 뒤집어쓴 채 돌아섰다
Processing article : 뇌사상태 빠진 70대 택시기사, 간 기증해 생명 나눔하고 하늘로
Processing article : 주택가 불 끄고 사라진 택배기사에 ‘감사장’
Processing article : 보복운전 이경 “대리기사 많이 연락...‘내가 한 것 같다’는 분도”
Processing article :  '또' 5·18 폄훼 기사 배포한 인천시의장, 한동훈 비판 기사도 공유
Processing article : 허식 인천시의장, 이번엔 '한동훈 비난 기사' 공유 논란
Processing article : 충북 도로보수원 사망사고 낸 60대 화물차 기사 집행유예
Processing article : ‘택배기사’ 변신한 한동훈… 택배 들고 국회 깜짝방문
Processing article : ‘온정 배송’ 택배기사…매달 쌀 5포대 놓고 사라진다

11번째 기사글부터 크롤링을 시작합니다.
Processing article : 필리핀 '서민의 발' 지프니 단계적 퇴출에 운전기사 반발
Processing article : 경남도, 법인택시 기사에 처우 개선비 매달 5만원씩 지급
Processing article : 택배기사, 허위로 ‘배달완료’ 연락 남겼다 고객에게 항의받자 “X까세요”
Processing article : IT기업·언론 다 가진 베니오프 "AI 훈련에 기사 콘텐츠 도난"
Processing article : 조선일보 '기사 5만건 학습' 생성AI 도입…기자들 반응은
Processing article : 한경·쿠팡 '비판 기사' 갈등… "팩트 쓴 것, 문제 없다"
Processing article : "연유라떼 한잔은 기사님 드리세요" 카페 온기 높인 사연

21번째 기사글부터 

In [16]:
# Dataset Length Check
print(f'Titles: {len(titles)}')
print(f'Dates: {len(dates)}')
print(f'Articles: {len(articles)}')
print(f'Article URLs: {len(article_urls)}')
print(f'Press Companies: {len(press_companies)}')
if not (len(titles) == len(dates) == len(articles) == len(article_urls) == len(press_companies)):
    raise ValueError('Dataset Length is not equal')

print(f'{len(errors)} errors occured')

Titles: 383
Dates: 383
Articles: 383
Article URLs: 383
Press Companies: 383
28 errors occured


In [17]:
# 각 데이터 종류별 list에 담아둔 전체 데이터를 DataFrame에 모으고 엑셀 파일로 저장합니다.
# 파일명을 result_연도월일_시분.csv 로 지정합니다.
article_df = pd.DataFrame({
    'title': titles,
    'date': dates,
    'document': articles,
    'link': article_urls,
    'press': press_companies,
    'category': categories
})

article_df.to_csv(f'result_from_{start_date}_end_{end_date}.csv', index=False, encoding='utf-8')

In [18]:
article_df.sort_values(by='date', ascending=True)

,title,date,document,link,press,category
219,[단독]바이오다인 “글로벌 파트너는 로슈”…연매출 2천억 거뜬,2024.01.15. 오전 10:03,이 기사는 2024년01월15일 09시02분에 팜이데일리 프리미엄 콘텐츠로 선공개 ...,https://n.news.naver.com/mnews/article/018/000...,이데일리,IT/과학
338,임기 완료 앞둔 박재욱 코스포 의장의 '마지막 경고' [긱스],2024.01.15. 오전 10:06,이 기사는 프리미엄 스타트업 미디어 플랫폼 한경 긱스에 게재된 기사입니다.박재욱 코...,https://n.news.naver.com/mnews/article/015/000...,한국경제,경제
251,작년 아파트 청약자 10명 중 6명 수도권 선택…쏠림 현상 더 심해졌다,2024.01.15. 오전 10:12,▲ 위 사진은 기사 내용과 관련이 없습니다.지난해 아파트 청약자 중 과반이 수도권에...,https://n.news.naver.com/mnews/article/055/000...,SBS,경제
376,"[속보]윤""경기 남부권 622조원 규모 반도체 메가클러스터…일자리 300만개 창출""",2024.01.15. 오전 10:13,[서울=뉴시스] 후속기사가 이어집니다▶ 네이버에서 뉴시스 구독하기▶ K-Artpr...,https://n.news.naver.com/mnews/article/003/001...,뉴시스,정치
371,"[속보]윤""반도체산업, 민생 풍요롭게 하고 많은 국민에 일자리 제공""",2024.01.15. 오전 10:13,[서울=뉴시스] 후속기사가 이어집니다▶ 네이버에서 뉴시스 구독하기▶ K-Artpr...,https://n.news.naver.com/mnews/article/003/001...,뉴시스,정치
...,...,...,...,...,...,...
177,"비엘, 최근 논란에 시총20% 증발...""애꿎은 투자자만 피해""",2024.01.17. 오후 2:21,[이데일리 김지완 기자] 최근 비엘(142760)이 논란에 휩싸이면서 일주일 만에 ...,https://n.news.naver.com/mnews/article/018/000...,이데일리,경제
13,"IT기업·언론 다 가진 베니오프 ""AI 훈련에 기사 콘텐츠 도난""",2024.01.17. 오후 2:35,"나델라 ""AI 개발 지속하며 부작용 함께 고려해야""AI저작권 문제엔 베니오프 vs ...",https://n.news.naver.com/mnews/article/029/000...,디지털타임스,IT/과학
270,"'강원동계청소년올림픽 빙상, 강원선수 1명'…신경호 강원교육감 ""빙상종목 계열화""",2024.01.17. 오후 2:36,"17일 신경호 강원도교육감이 기자 간담회에서 빙상종목의 초, 중, 고 계열화를 심도...",https://n.news.naver.com/mnews/article/079/000...,노컷뉴스,사회
123,"헬스케어 스타트업, CES 혁신상 최다 수상...하남돼지집, AI 셰프 도입[Gee...",2024.01.17. 오후 2:51,이 기사는 프리미엄 스타트업 미디어 플랫폼 한경 긱스에 게재된 기사입니다.한국경제신...,https://n.news.naver.com/mnews/article/015/000...,한국경제,경제


In [19]:
article_df.category.isna().sum()
# 카테고리 분류가 제대로 되지 않은 뉴스 기사는 없다.

0

In [20]:
article_df.count().sum()
# 792개의 기사가 있다.

2298